## Import our dataset
- our dataset consists of several files:
    - train.csv: our training data
    - test.csv: our testing data
    - sample_submissions.csv: A sample submission file in the correct format (columns key and fare_amount). This dummy file 'predicts' fare_amount to be $11.35 for all rows, which is the mean fare_amount from the training set.

In [1]:
import pandas as pd
import os

DATA_FILES_PATH = 'projectDataFiles/'

def import_training_dataset(file_path, row_limit):
    """
    function to import the dataset into a pandas dataframe.

    Takes a row limit to limit the number of rows read.
    """
    return pd.read_csv(file_path, nrows=row_limit)

# assign the dataset to the TRAIN Dataframe, right now we are only loading 1,000,000 rows (possibly chunk and feather to reduce loading time)
TRAIN = import_training_dataset(f'{DATA_FILES_PATH}train.csv', 1000000)

# show the head of the TRAIN dataset to see its columns
TRAIN.head()

def import_testing_dataset(file_path):
    """
    function to imort the testing dataset.
    
    Takes a row limit to limit the number of rows read.
    """
    return pd.read_csv(file_path)

TEST = import_testing_dataset(f'{DATA_FILES_PATH}test.csv')
# show the head of the TEST dataset to see its columns
print("TEST")
TEST.head()
    

TEST


,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


## Define a function to get the manhattan distance between two lat/long points
- manhattan distance should be fairly relistic to new york because of the ways that streets work there, but we might want to use the real travel distance between locations somehow

In [24]:
def geo_manhattan_distance(lat1, lat2, long1, long2):
    """
    returns the manhattan distance between two geo points
    """
    return abs(lat2 - lat1) + abs(long2 - long1)


# test it out
geo_manhattan_distance(TRAIN['pickup_latitude'].iloc[0], TRAIN['dropoff_latitude'].iloc[0],\
TRAIN['pickup_longitude'].iloc[0], TRAIN['dropoff_longitude'].iloc[0])

# PIP INSTALL GMAPS TO INSTALL GOOGLE MAPS
# run on command line and/or terminal: pip install gmaps
# NEW: pip install -U googlemaps

import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key='GOOGLE_API_KEY')

print(googlemaps)

#distance = gmaps.distance_matrix("Los Angeles", "Las Vegas")
distance = gmaps.distance_matrix((34.0522, -118.2437), (36.1699, -115.1398))
print(distance)

# Geocoding an address
#geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

# Look up an address with reverse geocoding
#reverse_geocode_result = gmaps.reverse_geocode((36.1699, -115.1398))
#print(reverse_geocode_result)

# Request directions via public transit
#now = datetime.now()
#directions_result = gmaps.directions("Sydney Town Hall",
                                    # "Parramatta, NSW",
                                    # mode="transit",
                                    # departure_time=now)

#print(directions_result)


<module 'googlemaps' from '//anaconda3/lib/python3.7/site-packages/googlemaps/__init__.py'>
{'destination_addresses': ['650 Iwilei Rd Las Vegas: 701. N, Las Vegas, NV 89101, United States'], 'origin_addresses': ['Los Angeles International Airport, Los Angeles, CA, Los Angeles, CA 90012, USA'], 'rows': [{'elements': [{'distance': {'text': '433 km', 'value': 433461}, 'duration': {'text': '4 hours 3 mins', 'value': 14599}, 'status': 'OK'}]}], 'status': 'OK'}
